In [2]:
import torch
from torch.nn import functional as F
import random
import pickle

import azureml
from azureml.core import Workspace, Datastore, Experiment, Run, Environment, ScriptRunConfig
from azureml.core.compute import ComputeTarget, AmlCompute, AksCompute, ComputeTarget
from azureml.train.dnn import PyTorch

from azureml.core.webservice import Webservice, AksWebservice, AciWebservice
from azureml.core.dataset import Dataset
from azureml.core.resource_configuration import ResourceConfiguration
from azureml.core.conda_dependencies import CondaDependencies 

# check core SDK version number
print("Azure ML SDK Version: ", azureml.core.VERSION)

Azure ML SDK Version:  1.53.0


In [3]:
ws = Workspace.from_config()
print("Workspace:{}".format(ws.name))

Workspace:NanoMedGPT


In [4]:
compute_target = ComputeTarget(workspace=ws, name="GPU-Cluster")
print("Compute Target:{}".format(compute_target.name))

Compute Target:GPU-Cluster


In [5]:
ds = Datastore.get_default(ws)
print("Datastore:{}".format(ds.name))

Datastore:workspaceblobstore


In [6]:
exp = Experiment(workspace=ws, name="LocalVsCode-NanoMedGPT")

In [7]:
""" Upload to cloud """
# from azureml.data.dataset_factory import FileDatasetFactory
# ret = FileDatasetFactory.upload_directory(
#     PROCESSED_DATA_PATH,
#     (ds, "encodings/"),
#     overwrite=True,
#     show_progress=True
# )

' Upload to cloud '

In [9]:
""" Get FileDataSetFactory from cloud """
from azureml.data.dataset_factory import FileDatasetFactory
path = (ds, "encodings/")
ret = FileDatasetFactory.from_files(path)

In [10]:
type(ret)

azureml.data.file_dataset.FileDataset

In [18]:
conda_spec_file = "C:\\Users\\Ben\\git\\nano_medgpt\\nano_medgpt_env2.yml"
pytorch_env = Environment.from_conda_specification("nano_medgpt", conda_spec_file)

In [19]:
src = ScriptRunConfig(
    source_directory= "C:\\Users\\Ben\\git\\nano_medgpt\\src\\",
    script = 'aml_gpt_train.py',
    arguments=['--data-folder', ret.as_mount()],
    compute_target=compute_target,
    environment=pytorch_env
)

In [20]:
run = exp.submit(src)

In [21]:
from azureml.widgets import RunDetails

RunDetails(run).show()

_UserRunWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', '…